## OULAD pipeline
- 6 tabela
- feature engineering i spajanje u jednu konacnu tabelu
- definisani makroi, code_module, code_presentation, cutoff day
- handling missling values, 0 za nove feature gde nema aktivnosti
- treniranje i evaluacija 3 razlicita modela
- optimizacija hiperparametara najboljeg
- train na jednom code_presentation, test na drugom
- threshold tuning
- model export

## StudentInfo + StudentRegistration

In [2]:
import pandas as pd
import numpy as np

student_info = pd.read_csv('studentInfo.csv')

In [3]:
print('StudentInfo:', student_info.shape)

StudentInfo: (32593, 12)


In [4]:
student_info.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass


In [5]:
student_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   code_module           32593 non-null  object
 1   code_presentation     32593 non-null  object
 2   id_student            32593 non-null  int64 
 3   gender                32593 non-null  object
 4   region                32593 non-null  object
 5   highest_education     32593 non-null  object
 6   imd_band              31482 non-null  object
 7   age_band              32593 non-null  object
 8   num_of_prev_attempts  32593 non-null  int64 
 9   studied_credits       32593 non-null  int64 
 10  disability            32593 non-null  object
 11  final_result          32593 non-null  object
dtypes: int64(3), object(9)
memory usage: 3.0+ MB


#### Pretvaramo kategoricke u numericke i uklanjamo originalne vrednosti

In [6]:
student_info["dropout"] = np.where(
    student_info["final_result"].isin(["Failed", "Withdrawn"]), 1, 0
)
student_info["disabled"] = np.where(student_info["disability"].eq("Y"), 1, 0)
student_info["female"] = np.where(student_info["gender"].eq("F"), 1, 0)

In [7]:
student_info = student_info.drop(
    columns=["disability", "final_result", "gender"],
    errors="ignore"
)

#### Filtriramo code module i code presentation po zeljenom modulu, prezentaciji i datumu do kog posmatramo aktivnosti studenata

In [8]:
print('Code_module:', student_info['code_module'].unique(),
      'Code_presentation:', student_info['code_presentation'].unique()
     )

Code_module: ['AAA' 'BBB' 'CCC' 'DDD' 'EEE' 'FFF' 'GGG'] Code_presentation: ['2013J' '2014J' '2013B' '2014B']


#### Prvobitno je treniran model sa code_presentation 2013J, zatim nismo filtrirali po code_presentation kako bismo testirali na drugom

In [9]:
selected_code_module = "BBB"
#selected_code_presentation = "2013J"
cutoff_day = 30

In [10]:
student_info = student_info[
    (student_info["code_module"] == selected_code_module)
#   & (student_info["code_presentation"] == selected_code_presentation)
].copy()

In [11]:
student_info.head()

,code_module,code_presentation,id_student,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,dropout,disabled,female
748,BBB,2013B,23629,East Anglian Region,Lower Than A Level,20-30%,0-35,2,60,0,0,1
749,BBB,2013B,25107,East Anglian Region,Lower Than A Level,20-30%,0-35,0,120,0,0,1
750,BBB,2013B,27891,Scotland,Lower Than A Level,0-10%,0-35,2,120,1,1,0
751,BBB,2013B,29144,South Region,Lower Than A Level,60-70%,0-35,0,120,0,0,0
752,BBB,2013B,31663,North Region,A Level or Equivalent,30-40%,35-55,0,60,0,0,0


#### Radimo isto sa StudentReg tabelom, samo sto je potrebno ukljuciti samo one studente koji su bili registrovani do cutoff dana

In [12]:
student_reg = pd.read_csv('studentRegistration.csv')

In [13]:
print('StudentReg:', student_reg.shape)

StudentReg: (32593, 5)


In [14]:
student_reg.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


In [15]:
student_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   code_module          32593 non-null  object 
 1   code_presentation    32593 non-null  object 
 2   id_student           32593 non-null  int64  
 3   date_registration    32548 non-null  float64
 4   date_unregistration  10072 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.2+ MB


In [16]:
student_reg = student_reg[
    (student_reg["code_module"] == selected_code_module) &
    #(student_reg["code_presentation"] == selected_code_presentation) &
    (student_reg["date_unregistration"].isna() |
    (student_reg["date_unregistration"] > cutoff_day))
].copy()

In [17]:
student_reg.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
748,BBB,2013B,23629,-47.0,NaN
749,BBB,2013B,25107,-73.0,NaN
750,BBB,2013B,27891,-58.0,153.0
751,BBB,2013B,29144,-24.0,NaN
752,BBB,2013B,31663,-60.0,NaN


#### Kreiramo novi atribut late registration

In [18]:
student_reg["late_registration"] = np.where(student_reg["date_registration"] <= 0, 0, 1)

In [19]:
student_reg = student_reg.drop(columns = ["date_unregistration"])

#### Spajamo tabele StudentInfo i StudentReg

In [20]:
merged = student_info.merge(
    student_reg,
    how="left",
    on=["id_student", "code_module", "code_presentation"]
)

In [21]:
merged["late_registration"] = merged["late_registration"].fillna(0)

In [22]:
merged.head()

,code_module,code_presentation,id_student,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,dropout,disabled,female,date_registration,late_registration
0,BBB,2013B,23629,East Anglian Region,Lower Than A Level,20-30%,0-35,2,60,0,0,1,-47.0,0.0
1,BBB,2013B,25107,East Anglian Region,Lower Than A Level,20-30%,0-35,0,120,0,0,1,-73.0,0.0
2,BBB,2013B,27891,Scotland,Lower Than A Level,0-10%,0-35,2,120,1,1,0,-58.0,0.0
3,BBB,2013B,29144,South Region,Lower Than A Level,60-70%,0-35,0,120,0,0,0,-24.0,0.0
4,BBB,2013B,31663,North Region,A Level or Equivalent,30-40%,35-55,0,60,0,0,0,-60.0,0.0


---

# StudentVle + Vle

In [23]:
student_vle = pd.read_csv("studentVle.csv")
vle = pd.read_csv("vle.csv")

In [24]:
student_vle.head(), vle.head()

(  code_module code_presentation  id_student  id_site  date  sum_click
 0         AAA             2013J       28400   546652   -10          4
 1         AAA             2013J       28400   546652   -10          1
 2         AAA             2013J       28400   546652   -10          1
 3         AAA             2013J       28400   546614   -10         11
 4         AAA             2013J       28400   546714   -10          1,
    id_site code_module code_presentation activity_type  week_from  week_to
 0   546943         AAA             2013J      resource        NaN      NaN
 1   546712         AAA             2013J     oucontent        NaN      NaN
 2   546998         AAA             2013J      resource        NaN      NaN
 3   546888         AAA             2013J           url        NaN      NaN
 4   547035         AAA             2013J      resource        NaN      NaN)

In [25]:
print('Shape student_vle:', student_vle.shape,
      '\nShape vle:', vle.shape
     )

Shape student_vle: (10655280, 6) 
Shape vle: (6364, 6)


In [26]:
print('NA week_from:', vle["week_from"].isna().sum(),
      '\nNA week_to:', vle["week_to"].isna().sum()
     )

NA week_from: 5243 
NA week_to: 5243


#### Izbacujemo week from i week to zbog velikog broja NA

In [27]:
vle = vle.drop(columns=["week_from", "week_to"], errors="ignore")

#### Spajanje tabela i filtriranje

In [28]:
vle_merged = student_vle.merge(
    vle,
    how="inner",
    on=["code_module", "code_presentation", "id_site"]
)

In [29]:
vle_merged = vle_merged[
    (vle_merged["code_module"] == selected_code_module) &
    #(vle_merged["code_presentation"] == selected_code_presentation) &
    (vle_merged["date"] <= cutoff_day)
].copy()

#### *Feature engineering*, ukupan broj klikova, aktivnih dana, intenzitet angazovanja, poslednji aktivan dan, koliko razlicitih tipova aktivnosti je korisceno, prosecan broj klikova i varijansa klikova

In [30]:
overall = (
    vle_merged
    .groupby(["code_module", "code_presentation", "id_student"], as_index=False)
    .agg(
        total_clicks=("sum_click", "sum"),
        days_active=("date", "nunique"),
        event_count=("date", "count"),
        last_active_day=("date", "max"),
        activity_type_count=("activity_type", "nunique"),
        avg_clicks=("sum_click", "mean"),
        var_clicks=("sum_click", "var")
    )
)
overall.head()

,code_module,code_presentation,id_student,total_clicks,days_active,event_count,last_active_day,activity_type_count,avg_clicks,var_clicks
0,BBB,2013B,23629,53,6,21,21,4,2.523810,4.261905
1,BBB,2013B,25107,85,8,30,29,6,2.833333,15.867816
2,BBB,2013B,27891,187,9,61,23,6,3.065574,14.862295
3,BBB,2013B,29144,212,10,84,20,7,2.523810,8.999426
4,BBB,2013B,31663,1193,39,363,30,8,3.286501,13.757469


In [31]:
overall[["total_clicks","days_active","event_count"]] = (
    overall[["total_clicks","days_active","event_count"]].fillna(0)
)

#### *Feature engineering*, pivot po tipu aktivnosti, ukupan broj klikova po tipu, angazovanje po tipu i ukupan broj aktivnih dana po tipu

In [32]:
by_type = (
    vle_merged.groupby(["code_module", "code_presentation", "id_student", "activity_type"], as_index=False)
      .agg(
          type_total_clicks=("sum_click", "sum"),
          type_event_count=("sum_click", "count"),
          type_days_active=("date", "nunique")
      )
)

In [33]:
pivot_clicks = by_type.pivot_table(
    index=["code_module", "code_presentation", "id_student"],
    columns="activity_type",
    values="type_total_clicks",
    aggfunc="sum"
)
pivot_clicks.columns = [f"{c}_total_clicks" for c in pivot_clicks.columns]

In [34]:
pivot_event_count = by_type.pivot_table(
    index=["code_module", "code_presentation", "id_student"],
    columns="activity_type",
    values="type_event_count",
    aggfunc="sum"
)
pivot_event_count.columns = [f"{c}_event_count" for c in pivot_event_count.columns]

In [35]:
pivot_days = by_type.pivot_table(
    index=["code_module", "code_presentation", "id_student"],
    columns="activity_type",
    values="type_days_active",
    aggfunc="sum"
)
pivot_days.columns = [f"{c}_days_active" for c in pivot_days.columns]

In [36]:
pivot_joined = pd.concat([pivot_clicks, pivot_event_count, pivot_days], axis=1).reset_index()

In [37]:
pivot_joined.head()

,code_module,code_presentation,id_student,forumng_total_clicks,glossary_total_clicks,homepage_total_clicks,oucollaborate_total_clicks,oucontent_total_clicks,ouelluminate_total_clicks,quiz_total_clicks,...,glossary_days_active,homepage_days_active,oucollaborate_days_active,oucontent_days_active,ouelluminate_days_active,quiz_days_active,resource_days_active,sharedsubpage_days_active,subpage_days_active,url_days_active
0,BBB,2013B,23629,32.0,NaN,14.0,NaN,NaN,NaN,NaN,...,NaN,6.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN
1,BBB,2013B,25107,57.0,NaN,18.0,NaN,1.0,NaN,NaN,...,NaN,8.0,NaN,1.0,NaN,NaN,1.0,NaN,2.0,2.0
2,BBB,2013B,27891,127.0,NaN,39.0,NaN,1.0,NaN,NaN,...,NaN,9.0,NaN,1.0,NaN,NaN,3.0,NaN,2.0,5.0
3,BBB,2013B,29144,86.0,1.0,57.0,NaN,3.0,NaN,NaN,...,1.0,9.0,NaN,3.0,NaN,NaN,5.0,NaN,6.0,6.0
4,BBB,2013B,31663,616.0,3.0,266.0,NaN,1.0,4.0,NaN,...,3.0,38.0,NaN,1.0,3.0,NaN,19.0,NaN,19.0,13.0


In [38]:
pivot_joined = pivot_joined.fillna(0)

In [39]:
merged2 = overall.merge(
    pivot_joined,
    how="left",
    on=["code_module", "code_presentation", "id_student"]
)

In [40]:
merged2.head()

,code_module,code_presentation,id_student,total_clicks,days_active,event_count,last_active_day,activity_type_count,avg_clicks,var_clicks,...,glossary_days_active,homepage_days_active,oucollaborate_days_active,oucontent_days_active,ouelluminate_days_active,quiz_days_active,resource_days_active,sharedsubpage_days_active,subpage_days_active,url_days_active
0,BBB,2013B,23629,53,6,21,21,4,2.523810,4.261905,...,0.0,6.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,BBB,2013B,25107,85,8,30,29,6,2.833333,15.867816,...,0.0,8.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,2.0
2,BBB,2013B,27891,187,9,61,23,6,3.065574,14.862295,...,0.0,9.0,0.0,1.0,0.0,0.0,3.0,0.0,2.0,5.0
3,BBB,2013B,29144,212,10,84,20,7,2.523810,8.999426,...,1.0,9.0,0.0,3.0,0.0,0.0,5.0,0.0,6.0,6.0
4,BBB,2013B,31663,1193,39,363,30,8,3.286501,13.757469,...,3.0,38.0,0.0,1.0,3.0,0.0,19.0,0.0,19.0,13.0


---

## StudentAssessment + Assessments

In [41]:
student_assessment = pd.read_csv("StudentAssessment.csv")
assessments = pd.read_csv("assessments.csv")

In [42]:
student_assessment.head(), assessments.head()

(   id_assessment  id_student  date_submitted  is_banked  score
 0           1752       11391              18          0   78.0
 1           1752       28400              22          0   70.0
 2           1752       31604              17          0   72.0
 3           1752       32885              26          0   69.0
 4           1752       38053              19          0   79.0,
   code_module code_presentation  id_assessment assessment_type   date  weight
 0         AAA             2013J           1752             TMA   19.0    10.0
 1         AAA             2013J           1753             TMA   54.0    20.0
 2         AAA             2013J           1754             TMA  117.0    20.0
 3         AAA             2013J           1755             TMA  166.0    20.0
 4         AAA             2013J           1756             TMA  215.0    30.0)

In [43]:
print('Shape student_assessment:', student_assessment.shape,
      '\nShape assessments:', assessments.shape
     )

Shape student_assessment: (173912, 5) 
Shape assessments: (206, 6)


In [44]:
assess_merged = student_assessment.merge(
    assessments,
    how="inner",
    on="id_assessment"
)

In [45]:
assess_merged = assess_merged[
    (assess_merged["code_module"] == selected_code_module) &
#    (assess_merged["code_presentation"] == selected_code_presentation) &
    (assess_merged["date"] <= cutoff_day)
].copy()

#### Zadrzali smo assignmente koji su do cutoff dana, i pravimo atribut late submittion kako bismo uhvatili one koji nisu predali do tada

In [46]:
assess_merged["late_submission"] = np.where(
    assess_merged["date_submitted"].isna() | (assess_merged["date_submitted"] > cutoff_day),1, 0
)

#### Atribut weighted score, koliko je svaki tip assessment-a vazan, assessment total weight je ukupna tezina assessment-a, a weighted score sum je ukupan ucinak studenta

In [47]:
assess_merged["weighted_score"] = assess_merged["score"] * assess_merged["weight"]

In [48]:
assessment_overall = (
    assess_merged
    .groupby(["code_module", "code_presentation", "id_student"], as_index=False)
    .agg(
        assessment_count=("id_assessment", "count"),
        assessment_avg_score=("score", "mean"),
        assessment_total_weight=("weight", "sum"),
        weighted_score_sum=("weighted_score", "sum")
    )
)

In [49]:
assessment_overall["assessment_count"] = (
    assessment_overall["assessment_count"].fillna(0)
)
assessment_overall["weighted_score_sum"] = (
    assessment_overall["weighted_score_sum"].fillna(0)
)

#### Prosecan score studenta po tipu assessmenta i koliko puta je student predao taj assessment

In [50]:
assess_by_type = (
    assess_merged
    .groupby(
        ["code_module", "code_presentation", "id_student", "assessment_type"],
        as_index=False)
    .agg(
        type_score_mean=("score", "mean"),
        type_submission_count=("id_assessment", "count")
    )
)

In [51]:
pivot_assess_scores = assess_by_type.pivot_table(
    index=["code_module", "code_presentation", "id_student"],
    columns="assessment_type",
    values="type_score_mean",
    aggfunc="sum"
)
pivot_assess_scores.columns = [f"{c}_avg_score" for c in pivot_assess_scores.columns]

In [52]:
pivot_assess_counts = assess_by_type.pivot_table(
    index=["code_module", "code_presentation", "id_student"],
    columns="assessment_type",
    values="type_submission_count",
    aggfunc="sum"
)
pivot_assess_counts.columns = [f"{c}_submission_count" for c in pivot_assess_counts.columns]

In [53]:
assessment_pivot = pd.concat([pivot_assess_scores, pivot_assess_counts], axis=1).reset_index()

In [54]:
assessment_pivot = assessment_pivot.fillna(0)

In [55]:
merged3 = assessment_overall.merge(
    assessment_pivot,
    how="left",
    on=["code_module", "code_presentation", "id_student"]
)

In [56]:
merged3.head()

,code_module,code_presentation,id_student,assessment_count,assessment_avg_score,assessment_total_weight,weighted_score_sum,TMA_avg_score,TMA_submission_count
0,BBB,2013B,23629,1,67.0,5.0,335.0,67.0,1
1,BBB,2013B,25107,1,59.0,5.0,295.0,59.0,1
2,BBB,2013B,27891,1,73.0,5.0,365.0,73.0,1
3,BBB,2013B,29144,1,60.0,5.0,300.0,60.0,1
4,BBB,2013B,31663,1,88.0,5.0,440.0,88.0,1


#### Za cutoff 15 ne ulaze assessment-i, nemaju signal za zadat kurs, zato sam koristila 30

---

## Spajanje sve tri tabele

In [57]:
df_final = merged.merge(
    merged2,
    how="left",
    on=["code_module","code_presentation","id_student"]
)

df_final = df_final.merge(
    merged3,
    how="left",
    on=["code_module","code_presentation","id_student"]
)

In [58]:
df_final.to_csv('df_final.csv')

---

## Preprocessing i modeling, izbor najboljeg algoritma

In [59]:
import pandas as pd

df_final = pd.read_csv('df_final.csv')

#### Za code module BBB i code presentation 2013J do 30. dana, najbolji rezultat daje model RandomForest sa AUC 0.860

## Hyperparameter tuning, trening na code_presentation 2013J, test na 2014J

In [60]:
train_pres = "2013J"
test_pres  = "2014J"

df_model = df_final.drop(columns = ["Unnamed: 0"], errors = 'ignore').copy()
drop_cols = ["dropout","id_student","code_module","code_presentation","date_unregistration"]
y_train = df_model.loc[df_model["code_presentation"] == train_pres, "dropout"].astype(int)
X_train = df_model.loc[df_model["code_presentation"] == train_pres].drop(columns=drop_cols, errors="ignore")

y_test  = df_model.loc[df_model["code_presentation"] == test_pres, "dropout"].astype(int)
X_test  = df_model.loc[df_model["code_presentation"] == test_pres].drop(columns=drop_cols, errors="ignore")

In [64]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

num_cols = X_train.select_dtypes(include=[np.number]).columns
cat_cols = X_train.select_dtypes(exclude=[np.number]).columns

numeric = Pipeline([
    ("imputer", SimpleImputer(strategy = "median")),
    ("scaler", StandardScaler(with_mean = False))
])

categorical = Pipeline([
    ("imputer", SimpleImputer(strategy = "most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown = "ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric, num_cols),
        ("cat", categorical, cat_cols)
    ],
    remainder = "drop"
)

rf_pipeline = Pipeline([
    ("preprocess", preprocess),
    ("model", RandomForestClassifier(
        random_state=42,
        class_weight="balanced_subsample",
        n_jobs=-1
    ))
])

param_dist = {
    "model__n_estimators": [300, 500, 800, 1200],
    "model__max_depth": [5, 10, 20, 30],
    "model__min_samples_split": [2, 5, 10, 20],
    "model__min_samples_leaf": [2, 4, 8]
}

search = RandomizedSearchCV(
    rf_pipeline,
    param_distributions=param_dist,
    n_iter=20,
    scoring="roc_auc",
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

search.fit(X_train, y_train)

best_rf = search.best_estimator_
print("Best CV AUC:", search.best_score_)
print("Best params:", search.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best CV AUC: 0.8489848112600029
Best params: {'model__n_estimators': 500, 'model__min_samples_split': 10, 'model__min_samples_leaf': 2, 'model__max_depth': 30}


In [65]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

proba = best_rf.predict_proba(X_test)[:, 1]
pred = (proba >= 0.5).astype(int)

print("AUC:", roc_auc_score(y_test, proba))
print("Precision:", precision_score(y_test, pred))
print("Recall:", recall_score(y_test, pred))
print("F1:", f1_score(y_test, pred))

AUC: 0.8376128205505375
Precision: 0.9392624728850325
Recall: 0.5781041388518025
F1: 0.715702479338843


#### Treshold tuning za Recall

In [66]:
import numpy as np

thresholds = np.linspace(0.1,0.9,17)

for t in thresholds:
    pred = (proba >= t).astype(int)
    print(t, 'Recall:', recall_score(y_test, pred), 'Precision:', precision_score(y_test,pred))

0.1 Recall: 1.0 Precision: 0.3267888307155323
0.15000000000000002 Recall: 0.9986648865153538 Precision: 0.3270660253607346
0.2 Recall: 0.9893190921228304 Precision: 0.33228699551569507
0.25 Recall: 0.9238985313751669 Precision: 0.3607924921793535
0.30000000000000004 Recall: 0.822429906542056 Precision: 0.5095119933829612
0.35 Recall: 0.7022696929238985 Precision: 0.7460992907801418
0.4 Recall: 0.6275033377837116 Precision: 0.8671586715867159
0.45000000000000007 Recall: 0.5914552736982643 Precision: 0.9134020618556701
0.5 Recall: 0.5781041388518025 Precision: 0.9392624728850325
0.55 Recall: 0.562082777036048 Precision: 0.9767981438515081
0.6 Recall: 0.5500667556742324 Precision: 0.9903846153846154
0.65 Recall: 0.5353805073431241 Precision: 0.9950372208436724
0.7000000000000001 Recall: 0.5113484646194927 Precision: 0.9973958333333334
0.75 Recall: 0.4819759679572764 Precision: 0.9972375690607734
0.8 Recall: 0.4672897196261682 Precision: 1.0
0.85 Recall: 0.4512683578104139 Precision: 1.0
0

In [67]:
best_threshold = 0.35

## Deployment

In [68]:
import joblib

bundle = {
    "model": best_rf,
    "threshold": best_threshold
}

joblib.dump(bundle, "final_oulad_pipeline.joblib")

['final_oulad_pipeline.joblib']